In [16]:
from pathlib import Path 
import os, dotenv
dotenv.load_dotenv()
os.chdir(Path(os.getenv("PYTHONPATH")).expanduser())

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

In [ ]:
VERSION = "1.2s"
VARIATION = "nocomments"
#jsonl_path = f"data/training_datasets/verified_{VARIATION}.jsonl"
#jsonl_path = "data/training_datasets/verified_nocomments.jsonl"

jsonl_path = "data/some_sources_raw.jsonl"

df = pd.read_json(jsonl_path, lines=True)

In [19]:
df.head()

,filename,filetype,content,variant
0,AES-GCM-SIV-proof/proof/cryptol-specs/AES.cry,cry,module AES where\n\nimport `Common::AES\n\ntyp...,without_comments
1,AES-GCM-SIV-proof/proof/cryptol-specs/AES128.cry,cry,module AES128 where\n\nimport `Common::AES\nim...,without_comments
2,AES-GCM-SIV-proof/proof/cryptol-specs/intrinsi...,cry,module Intrinsics where\n\nimport `Common::AES...,without_comments
3,AES-GCM-SIV-proof/proof/cryptol-specs/AES256.cry,cry,module AES256 where\n\nimport `Common::AES\nim...,without_comments
4,AES-GCM-SIV-proof/proof/cryptol-specs/TBox.cry,cry,type Nb = 4\ntype State = [4][Nb]...,without_comments


In [ ]:
from src.preprocessing.interpreter_process import verify_df_row_with_cryptol
from src.preprocessing.saw_subprocess import run_saw_script, load_saw_results

saw_results = []
MOUNT_DIR = os.getenv("MOUNT_DIR")
dir_path = Path(MOUNT_DIR)
dir_path.mkdir(parents=True, exist_ok=True) 
SERVER_URL = "http://localhost:8080"
#print(verify_df_row_with_cryptol(df, 35, host_mount_dir=MOUNT_DIR, server_url=SERVER_URL))
rows = []
out_path = Path(f"data/training_datasets/syntax_check/cry_syntax_verification_results_{VERSION}.jsonl")
out_path.parent.mkdir(parents=True, exist_ok=True)  # create dirs if missing
#results_df = load_saw_results(f"data/training_datasets/syntax_check/cry_syntax_verification_results_{VERSION}.jsonl")
for i in range(len(df)):
    if i % 10 == 0 and i > 0:
        print(f"Processed {i} of {len(df)} files")
    if df.iloc[i]["filetype"] == 'cry':
        repl_result = verify_df_row_with_cryptol(
                df, i, host_mount_dir=MOUNT_DIR, server_url=SERVER_URL)
        rows.append({"filename" : df.iloc[i]["filename"], **repl_result["load_info"]})
    '''
    else:
        if df.iloc[i]["filename"] in results_df['filename'].values:
            result_ = results_df[results_df['filename'] == df.iloc[i]["filename"]]
            rows.append({"filename" : df.iloc[i]["filename"], "load_ok": result_["load_ok"].values[0], "error": result_["error"].values[0] if not result_["load_ok"].values[0] else None, "file_deps": ["SAW"]})
        else:
            rows.append({"filename" : df.iloc[i]["filename"], "load_ok": False, "error": "saw", "file_deps": ["SAW"]})
            continue
            saw_result = run_saw_script(f"{os.getenv("REPO_ROOT")}/{df.iloc[i]['filename']}")
            row = {"filename": df.iloc[i]["filename"], **saw_result}
            results_df = pd.concat([results_df, pd.DataFrame([row])], ignore_index=True)
            #saw_results.append({"filename": df.iloc[i]["filename"], **saw_result})
            if saw_result["returncode"] == 0:
                rows.append({"filename" : df.iloc[i]["filename"], "load_ok": True, "error": None, "file_deps": ["SAW"]})
            else:
                rows.append({"filename" : df.iloc[i]["filename"], "load_ok": False, "error": "saw", "file_deps": ["SAW"]})
    '''
    
results_df = pd.DataFrame(rows)
results_df.to_json("data/training_datasets/syntax_check/cry_saw_syntax_verification_results.jsonl", lines=True, orient="records")

Processed 10 of 692 files
Processed 20 of 692 files
Processed 30 of 692 files
Processed 40 of 692 files
Processed 50 of 692 files
Processed 60 of 692 files
Processed 70 of 692 files
Processed 80 of 692 files
Processed 90 of 692 files
Processed 100 of 692 files
Processed 110 of 692 files
Processed 120 of 692 files
Processed 130 of 692 files
Processed 140 of 692 files
Processed 150 of 692 files
Processed 160 of 692 files
Processed 170 of 692 files
Processed 180 of 692 files
Processed 190 of 692 files
Processed 200 of 692 files
Processed 210 of 692 files
Processed 220 of 692 files
Processed 230 of 692 files
Processed 240 of 692 files
Processed 250 of 692 files
Processed 260 of 692 files
Processed 270 of 692 files
Processed 280 of 692 files
Processed 290 of 692 files
Processed 300 of 692 files
Processed 310 of 692 files
Processed 320 of 692 files
Processed 330 of 692 files
Processed 340 of 692 files
Processed 350 of 692 files
Processed 360 of 692 files
Processed 370 of 692 files
Processed 

In [21]:
results_df['repo'] = results_df['filename'].apply(lambda x: x.split('/')[0])

results_df.head()

,filename,load_ok,file,error,file_deps,repo
0,AES-GCM-SIV-proof/proof/cryptol-specs/AES.cry,True,files/AES_5191evb3.cry,None,"[Common::AES, Cryptol]",AES-GCM-SIV-proof
1,AES-GCM-SIV-proof/proof/cryptol-specs/AES128.cry,True,files/AES128__3uait8s.cry,None,"[AES, Common::AES, Cryptol]",AES-GCM-SIV-proof
2,AES-GCM-SIV-proof/proof/cryptol-specs/intrinsi...,False,files/intrinsics_jugi0u7q.cry,Parse error at files/intrinsics_jugi0u7q.cry:1...,NaN,AES-GCM-SIV-proof
3,AES-GCM-SIV-proof/proof/cryptol-specs/AES256.cry,True,files/AES256_7m7blh3x.cry,None,"[AES, Common::AES, Cryptol]",AES-GCM-SIV-proof
4,AES-GCM-SIV-proof/proof/cryptol-specs/TBox.cry,True,files/TBox_0tvla3br.cry,None,[Cryptol],AES-GCM-SIV-proof


In [32]:
repl_results_df = pd.DataFrame(rows)
repl_results_df = repl_results_df.join(df[["filename", "filetype"]].set_index('filename'), on='filename', how='left', rsuffix='_orig')
repl_results_df["repo"] = repl_results_df["filename"].apply(lambda x: x.split('/')[0])
passed = repl_results_df[repl_results_df["load_ok"] == True]
for repo in passed['repo'].unique():
    print(f"Repo: {repo}, Files: {len(passed[passed['repo'] == repo])}")

Repo: AES-GCM-SIV-proof, Files: 14
Repo: aws-lc-verification, Files: 33
Repo: BLST-Verification, Files: 5
Repo: ckzg-eip-4844-verification, Files: 2
Repo: cryptol, Files: 156
Repo: cryptol-specs, Files: 146
Repo: formal-verso, Files: 2
Repo: hand_crafted, Files: 23
Repo: SAW-course, Files: 23
Repo: saw-demos, Files: 4
Repo: saw-script, Files: 28


In [30]:
failed = repl_results_df[repl_results_df["load_ok"] == False]
for repo in failed['repo'].unique():
    print(f"Repo: {repo}, Files: {len(failed[failed['repo'] == repo])}")

Repo: AES-GCM-SIV-proof, Files: 8
Repo: aws-lc-verification, Files: 51
Repo: BLST-Verification, Files: 49
Repo: ckzg-eip-4844-verification, Files: 10
Repo: cryptol, Files: 46
Repo: cryptol-specs, Files: 43
Repo: formal-verso, Files: 9
Repo: hand_crafted, Files: 3
Repo: SAW-course, Files: 16
Repo: saw-demos, Files: 7
Repo: saw-script, Files: 14


In [35]:
lines = []

for _, row in failed.iterrows():
    lines.append(str(row["filename"]))   # ensure it's a string

# newline-separated string
filenames_str = "\n".join(lines)

# optional: write to a text file
with open("syntax_verification_failures.txt", "w", encoding="utf-8") as f:
    f.write(filenames_str)


In [25]:
print(len(repl_results_df[(repl_results_df["load_ok"] == True) & (repl_results_df["filetype"] == "cry")]))
print(len(repl_results_df[(repl_results_df["load_ok"] == False) & (repl_results_df["filetype"] == "cry")]))

436
256


In [26]:
print(len(repl_results_df[(repl_results_df["load_ok"] == True) & (repl_results_df["filetype"] != "cry")]))
print(len(repl_results_df[(repl_results_df["load_ok"] == False) & (repl_results_df["filetype"] != "cry")]))

0
0


In [27]:
print(len(repl_results_df[repl_results_df["load_ok"] == True]))
print(len(repl_results_df[repl_results_df["load_ok"] == False]))

436
256
